In [1]:
import pandas as pd

In [21]:
ekb = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\EKB_places_with_description.csv')
nn = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\NN_places_with_description.csv')
vlad = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Vladimir_places_with_description.csv')
yar = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Yaroslavl_places_with_description.csv')

### object

In [108]:
col = ['XID', 'Name', 'City', 'Lon', 'Lat', 'Rate', 'description']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [109]:
final = final.rename(columns={'XID': 'id', 'Name': 'name', 'City':'city', 'Lon':'lon', 'Lat':'lat', 'Rate':'rate'})

In [ ]:
final.drop_duplicates()

In [21]:
final.to_csv(r'D:\ai-sightseeing-nto\backend\data\final_db.csv')

### kind

In [112]:
col = ['Kind']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [105]:
all_cat = []

for i in range(len(final)):
    all_cat.extend(final.iloc[i]['Kind'].split(','))

all_cat_set = set(all_cat)


In [106]:
kind_list = sorted(list(all_cat_set))
id_list = range(1, len(kind_list) + 1)

df = pd.DataFrame({'id': id_list, 'type': kind_list})

In [34]:
df.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\kind_db.csv', index=False)

### kind obj

In [107]:
df

,id,type
0,1,accomodations
1,2,archaeological_museums
2,3,archaeology
3,4,architecture
4,5,art_galleries
...,...,...
90,91,urban_environment
91,92,view_points
92,93,war_memorials
93,94,water_towers


In [113]:
col = ['Kind', 'XID']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [114]:
final

,Kind,XID
0,"sport,architecture,historic_architecture,inter...",W38411380
1,"architecture,historic_architecture,interesting...",N2885181131
2,"architecture,historic_architecture,interesting...",W38581890
3,"architecture,historic_architecture,interesting...",N1930476141
4,"architecture,historic_architecture,interesting...",N3002726097
...,...,...
95,"historic,archaeology,interesting_places,other_...",Q97161882
96,"historic_architecture,architecture,interesting...",Q4091101
97,"museums,cultural,science_museums,children_muse...",U3739
98,"biographical_museums,museums,cultural,interest...",U3885


In [118]:
df_exploded = final.assign(Kind=final['Kind'].str.split(',')).explode('Kind').rename(columns={'Kind': 'type'})

In [119]:
df_exploded

,type,XID
0,sport,W38411380
0,architecture,W38411380
0,historic_architecture,W38411380
0,interesting_places,W38411380
0,stadiums,W38411380
...,...,...
99,museums,U4106
99,cultural,U4106
99,interesting_places,U4106
99,local_museums,U4106


In [122]:
last = pd.merge(df_exploded, df, on = 'type')

In [123]:
df_id_xid = pd.DataFrame({'id': range(len(last)), 'id_obj': last['XID'], 'id_kind':last['id']})

In [124]:
df_id_xid.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\obj_kind_db.csv', index=False)

In [125]:
df_id_xid

,id,id_obj,id_kind
0,0,W38411380,82
1,1,W587989335,82
2,2,R2767108,82
3,3,W38411380,4
4,4,N2885181131,4
...,...,...,...
1798,1798,W222984781,46
1799,1799,Q2494278,3
1800,1800,Q97161882,3
1801,1801,Q2494278,62


### photo

In [ ]:
col = ['Kind']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [20]:
ekbph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\EKB_images.csv')
nnph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\NN_images.csv')
vladph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Vladimir_images.csv')
yarph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Yaroslavl_images.csv')

In [22]:
ekbph = ekbph.drop_duplicates(subset = ['image'])
nnph = nnph.drop_duplicates(subset = ['image'])
vladph = vladph.drop_duplicates(subset = ['image'])
yarph = yarph.drop_duplicates(subset = ['img'])

ekb = ekb.drop_duplicates(subset = ['Name'])
nn = nn.drop_duplicates(subset = ['Name'])
vlad = vlad.drop_duplicates(subset = ['Name'])
yar = yar.drop_duplicates(subset = ['Name'])

yar.rename(columns={'img': 'image'}, inplace=True)


In [23]:
ekb.rename(columns={'Name': 'name'}, inplace=True)
ekbph = pd.merge(ekb, ekbph, on = 'name')

nn.rename(columns={'Name': 'name'}, inplace=True)
nnph = pd.merge(nn, nnph, on = 'name')

vlad.rename(columns={'Name': 'name'}, inplace=True)
vladph = pd.merge(vlad, vladph, on = 'name')

yar.rename(columns={'Name': 'name'}, inplace=True)
yarph = pd.merge(yar, yarph, on = 'name')

In [25]:
yarph.rename(columns={'img': 'image'}, inplace=True)

In [26]:
col = ['XID', 'image']
df = pd.concat([ekbph[col], nnph[col], vladph[col], yarph[col]])

In [27]:
df = df.drop_duplicates()
df = df.rename(columns={'XID':'id_obj', 'image':'base64'})

In [33]:
counts = df['base64'].value_counts()
duplicates = counts[counts > 1].index

df = df[~df['base64'].isin(duplicates)]


In [34]:
df

,id_obj,base64
0,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
1,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
2,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
...,...,...
3958,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3959,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3960,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3961,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...


In [31]:
df['base64'].unique().shape

(13281,)

In [35]:
df.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\photo_obj_db.csv', index=False)

In [89]:
df_id_xid = pd.DataFrame({'XID': df['XID'], 'base64': df['image']})
 
df_id_image = pd.DataFrame({'id': range(len(list(set(df['image'].to_list())))), 'base64': list(set(df['image'].to_list()))})

In [91]:
df_id_xid = pd.merge(df_id_xid, df_id_image, on = 'base64')

In [96]:
df_id_xid = df_id_xid[['XID', 'id']].drop_duplicates()

In [97]:
df_id_xid = pd.DataFrame({'id': range(len(df_id_xid)), 'id_obj': df_id_xid['XID'], 'id_photo':df_id_xid['id']})

In [98]:
df_id_xid.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\obj_photo_db.csv', index=False)

In [86]:
df_id_image.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\photo_db.csv', index=False)

In [90]:
df_id_image

,id,base64
0,0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
1,1,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
2,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3,3,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4,4,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
...,...,...
13276,13276,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
13277,13277,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
13278,13278,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
13279,13279,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
